In [1]:
1+1


2

In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

Top k = searches content char max: 

In [25]:
%pip install wikipedia

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11706 sha256=0a61f810327e9a539c41f8767e45143d93b29c2dbc3f548d281dedf28137d426
  Stored in directory: c:\users\fazal\appdata\local\pip\cache\wheels\63\47\7c\a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia
Note: you may need to restart the kernel to use updated packages.


In [13]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200 )
tool=WikipediaQueryRun(api_wrapper=api_wrapper)

In [14]:
tool.name

'wikipedia'

In [15]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
embed=HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")
from langchain_text_splitters import RecursiveCharacterTextSplitter


d:\LangChain\Chatbot\chatbotLLm\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
d:\LangChain\Chatbot\chatbotLLm\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [18]:
laoder=WebBaseLoader("https://python.langchain.com/v0.1/docs")
docs=laoder.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,embed)
retriver=vectordb.as_retriever() 


In [20]:
from langchain.tools.retriever import create_retriever_tool

langTool=create_retriever_tool(retriver,"langsmithsearch",
                      "Search for information about Langsmith.For any question about Langsmith , you must use this tool  ")

In [21]:
langTool.name

'langsmithsearch'

In [23]:
%pip install arxiv


  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/81.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/81.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/81.1 kB ? eta -:--:--
   --------------- ------------------------ 30.7/81.1 kB ? eta -:--:--
   ------------------------------ --------- 61.4/81.1 kB 1.1 MB/s eta 0:00:01
   ------------------------------ --------- 61.4/81.1 kB 1.1 MB/s eta 0:00:01
   ------------------------------ --------- 61.4/81.1 kB 1.1 MB/s eta 0:00:01
   ----------------------------------- ---- 71.7/81.1 kB 438.9 kB/s eta 0:00:01
   ---------------------------------------- 81.1/81.1 kB 378.9 kB/s eta 0:00:00
Using cached requests-2.31.0-py3-none-any.whl (62 kB)
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6060 sha256=552770aabbb4

from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun
arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)





Combing all the 3 tools in list 

In [26]:
tools=[tool,arxiv_wrapper,langTool]

Agents 

In [27]:
from langchain.agents import create_openai_tools_agent
import os 
from langchain_groq import ChatGroq
groq_api_key=os.getenv('GROQ_API_KEY')
llm=ChatGroq(groq_api_key=groq_api_key,
            
             model_name="Llama3-8b-8192")

In [28]:
from langchain import hub 

In [30]:
%pip install langchainhub

Note: you may need to restart the kernel to use updated packages.


In [31]:
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [32]:
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

ValueError: Unsupported function

arxiv_search=<class 'arxiv.Search'> arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>) top_k_results=1 ARXIV_MAX_QUERY_LENGTH=300 continue_on_failure=False load_max_docs=100 load_all_available_meta=False doc_content_chars_max=200 arxiv_result=<class 'arxiv.Result'>

Functions must be passed in as Dict, pydantic.BaseModel, or Callable. If they're a dict they must either be in OpenAI function format or valid JSON schema with top-level 'title' and 'description' keys.

In [ ]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

In [ ]:

agent_executor.invoke({"input":"Tell me about Langsmith"})